In [ ]:
!pip install tensorflow==2.3.0

In [ ]:
import numpy as np
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train, x_test = x_train.reshape(x_train.shape[0], 28*28), x_test.reshape(x_test.shape[0], 28*28)

In [14]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(200, activation='relu', input_shape=(28*28,)),
  tf.keras.layers.Dense(200, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax'),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=5)

In [ ]:
model.evaluate(x_test,  y_test, verbose=2)

In [19]:
model.save("network.h5")

In [ ]:
model.save("network")
converter = tf.lite.TFLiteConverter.from_saved_model("/content/network")
model_no_quant_tflite = converter.convert()


open("network.tflite", "wb").write(model_no_quant_tflite)

In [ ]:
def representative_dataset():
  for i in range(500):
    yield([x_train[i].reshape(1, 784).astype(np.float32)])

converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset
model_tflite = converter.convert()

open("network_with_quant.tflite", "wb").write(model_tflite)